In [ ]:
"""
Created on Sun Oct 21 2018
@author: Kimin Lee
"""

import import_ipynb
import torch
import numpy as np
import os
import lib_regression
import random

from sklearn.linear_model import LogisticRegressionCV

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def regression(id_dataset='', ood_dataset='', file_path='', score=''):    
    
    # train and measure the performance of Mahalanobis detector
    list_best_results, list_best_results_index = [], []
    list_best_results_out, list_best_results_index_out = [], []
    best_tnr, best_result, best_index = 0, 0, 0
    
    total_X, total_Y = lib_regression.load_characteristics(score, id_dataset, ood_dataset, file_path)
    X_val, Y_val, X_test, Y_test = lib_regression.block_split(total_X, total_Y)
        
    X_train = np.concatenate((X_val[:500], X_val[1000:1500]))
    Y_train = np.concatenate((Y_val[:500], Y_val[1000:1500]))
    X_val_for_test = np.concatenate((X_val[500:1000], X_val[1500:]))
    Y_val_for_test = np.concatenate((Y_val[500:1000], Y_val[1500:]))
    
    lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
    
    # y_pred = lr.predict_proba(X_train)[:, 1]
    # #print('training mse: {:.4f}'.format(np.mean(y_pred - Y_train)))
    # y_pred = lr.predict_proba(X_val_for_test)[:, 1]
    # #print('test mse: {:.4f}'.format(np.mean(y_pred - Y_val_for_test)))
    
    results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, file_path)
    
    if best_tnr < results['TMP']['TNR']:
        best_tnr = results['TMP']['TNR']
        best_index = score
        best_result = lib_regression.detection_performance(lr, X_test, Y_test, file_path)
            
    list_best_results_out.append(best_result)
    list_best_results_index_out.append(best_index)
    list_best_results.append(list_best_results_out)
    list_best_results_index.append(list_best_results_index_out)
        
    # print the results
    count_in = 0
    mtypes = ['TNR', 'AUROC', 'DTACC', 'AUIN', 'AUOUT']

    for in_list in list_best_results:
        out_list = ['svhn']
        count_out = 0
        for results in in_list:
            print('out_distribution: '+ out_list[count_out])
            for mtype in mtypes:
                print(' {mtype:6s}'.format(mtype=mtype), end='')
            print('\n{val:6.2f}'.format(val=100.*results['TMP']['TNR']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['AUROC']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['DTACC']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['AUIN']), end='')
            print(' {val:6.2f}\n'.format(val=100.*results['TMP']['AUOUT']), end='')
            print('Input noise: ' + list_best_results_index[count_in][count_out])
            print('')
            count_out += 1
        count_in += 1
        
    # 최종 AUROC, FPR(TNR 기반) 추출
    final_auroc = best_result['TMP']['AUROC']
    final_fpr = 1.0 - best_result['TMP']['TNR']
    
    return final_auroc, final_fpr

In [ ]:
def regression_fi(id_dataset='', ood_dataset='', file_path='', score=''):    
    
    # train and measure the performance of Mahalanobis detector
    list_best_results, list_best_results_index = [], []
    list_best_results_out, list_best_results_index_out = [], []
    best_tnr, best_result, best_index = 0, 0, 0
    
    total_X, total_Y = lib_regression.load_characteristics(score, id_dataset, ood_dataset, file_path)
    X_val, Y_val, X_test, Y_test = lib_regression.block_split(total_X, total_Y)
        
    X_train = np.concatenate((X_val[:500], X_val[1000:1500]))
    Y_train = np.concatenate((Y_val[:500], Y_val[1000:1500]))
    X_val_for_test = np.concatenate((X_val[500:1000], X_val[1500:]))
    Y_val_for_test = np.concatenate((Y_val[500:1000], Y_val[1500:]))
    
    lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
    
    # y_pred = lr.predict_proba(X_train)[:, 1]
    # #print('training mse: {:.4f}'.format(np.mean(y_pred - Y_train)))
    # y_pred = lr.predict_proba(X_val_for_test)[:, 1]
    # #print('test mse: {:.4f}'.format(np.mean(y_pred - Y_val_for_test)))
    
    results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, file_path)
    
    if best_tnr < results['TMP']['TNR']:
        best_tnr = results['TMP']['TNR']
        best_index = score
        best_result = lib_regression.detection_performance(lr, X_test, Y_test, file_path)
            
    list_best_results_out.append(best_result)
    list_best_results_index_out.append(best_index)
    list_best_results.append(list_best_results_out)
    list_best_results_index.append(list_best_results_index_out)
        
    # print the results
    count_in = 0
    mtypes = ['TNR', 'AUROC', 'DTACC', 'AUIN', 'AUOUT']

    for in_list in list_best_results:
        out_list = ['svhn']
        count_out = 0
        for results in in_list:
            print('out_distribution: '+ out_list[count_out])
            for mtype in mtypes:
                print(' {mtype:6s}'.format(mtype=mtype), end='')
            print('\n{val:6.2f}'.format(val=100.*results['TMP']['TNR']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['AUROC']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['DTACC']), end='')
            print(' {val:6.2f}'.format(val=100.*results['TMP']['AUIN']), end='')
            print(' {val:6.2f}\n'.format(val=100.*results['TMP']['AUOUT']), end='')
            print('Input noise: ' + list_best_results_index[count_in][count_out])
            print('')
            count_out += 1
        count_in += 1
        
    # 최종 AUROC, FPR(TNR 기반) 추출
    fi_final_auroc = best_result['TMP']['AUROC']
    fi_final_fpr = 1.0 - best_result['TMP']['TNR']
    
    return fi_final_auroc, fi_final_fpr    